In [1]:
import torch
import os
import numpy as np
from randmix import RandMix, SimpleAutoencoder, AdaIN
from top2 import Top2PseudoLabeling
from pca import DomainAlignmentModel, FeatureExtractor, LinearLayerPrototypes
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, TensorDataset

Total loss: 2.8574328422546387
Loss components: {'ce_loss': 2.358193874359131, 'contrastive_loss': 0.9984779357910156, 'distill_loss': 0.0}


In [2]:
train_dir = os.path.join('dataset', 'part_one_dataset', 'train_data')
eval_dir = os.path.join('dataset', 'part_one_dataset', 'eval_data')
train_path = os.path.join(train_dir, '1_train_data.tar.pth')
eval_path = os.path.join(eval_dir, '1_eval_data.tar.pth')

t = torch.load(train_path, weights_only = False)

In [3]:
t.keys()

dict_keys(['data', 'targets'])

In [6]:
randmix = RandMix()
pca = DomainAlignmentModel(feature_dim=128, num_classes=20)
t2pl = Top2PseudoLabeling()

TypeError: Top2PseudoLabeling.__init__() missing 1 required positional argument: 'model'

In [ ]:
class LWPModel(torch.nn.Module):
    def __init__(self, feature_dim, num_classes):
        super(LWPModel, self).__init__()
        self.feature_dim = feature_dim
        self.num_classes = num_classes
        self.randmix = RandMix()
        self.pca = PCA()
        self.t2pl = Top2PseudoLabeling()
        # Feature extractor and classifier
        self.feature_extractor = torch.nn.Sequential(
            torch.nn.Conv2d(3, 64, 3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2, 2),
            torch.nn.Conv2d(64, 128, 3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2, 2),
            torch.nn.Flatten(),
            torch.nn.Linear(128 * 8 * 8, feature_dim)
        )
        self.classifier = torch.nn.Linear(feature_dim, num_classes, bias=False)

    def forward(self, x):
        features = self.feature_extractor(x)
        return self.classifier(features)

# Function to load datasets
def load_data(path):
    data_dict = torch.load(path)
    images = torch.tensor(data_dict['data']).float()
    targets = torch.tensor(data_dict.get('targets', torch.zeros(images.size(0)).long()))  # Zero labels if no targets
    return TensorDataset(images, targets)

# Training, pseudo-labeling, and evaluation setup
feature_dim = 256
num_classes = 10  # Adjust as needed
num_models = 20

models = []
for i in range(num_models):
    print(f"Training model {i+1}/{num_models}")
    
    # Initialize model
    model = LWPModel(feature_dim, num_classes).cuda()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    train_path = f"{train_dir}/{i+1}_train_data.tar.pth"
    eval_path = f"{eval_dir}/{i+1}_eval_data.tar.pth"
    
    # Load dataset
    dataset = load_data(train_path)
    dataloader = DataLoader(dataset, batch_size=64, shuffle=True)
    
    # Supervised training for the first dataset with true labels
    if i == 0:
        for epoch in range(10):
            model.train()
            for images, labels in dataloader:
                images, labels = images.cuda(), labels.cuda()
                # RandMix augmentation
                augmented_images = randmix(images)
                optimizer.zero_grad()
                logits = model(augmented_images)
                loss = torch.nn.CrossEntropyLoss()(logits, labels)
                loss.backward()
                optimizer.step()
        print("Initial supervised model trained.")

    # Generate pseudo labels for unlabeled datasets
    else:
        model.eval()
        pseudo_labels = t2pl.generate_pseudo_labels(model, dataloader, num_classes)
        pseudo_dataset = TensorDataset(dataset[:][0], pseudo_labels)  # Replace targets with pseudo labels
        pseudo_dataloader = DataLoader(pseudo_dataset, batch_size=64, shuffle=True)

        # Train with pseudo labels using PCA for alignment
        for epoch in range(10):
            model.train()
            for images, pseudo_labels in pseudo_dataloader:
                images, pseudo_labels = images.cuda(), pseudo_labels.cuda()
                # RandMix augmentation
                augmented_images = randmix(images)
                optimizer.zero_grad()
                logits = model(augmented_images)
                
                # Compute losses
                ce_loss = torch.nn.CrossEntropyLoss()(logits, pseudo_labels)
                features = model.feature_extractor(images)
                pca_loss = pca(features, pseudo_labels, model.classifier.weight)
                total_loss = ce_loss + 0.5 * pca_loss  # Weighted loss
                total_loss.backward()
                optimizer.step()
        print(f"Model {i+1} trained with pseudo labels.")

    # Save trained model for later analysis or reuse
    models.append(model)

    # Evaluation
    eval_dataset = load_data(eval_path)
    eval_loader = DataLoader(eval_dataset, batch_size=64, shuffle=False)
    correct, total = 0, 0
    model.eval()
    with torch.no_grad():
        for images, labels in eval_loader:
            images, labels = images.cuda(), labels.cuda()
            logits = model(images)
            _, predicted = logits.max(1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Evaluation Accuracy for model {i+1}: {accuracy:.2f}%")

In [3]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

In [4]:
images = t['data'].reshape(-1, 3, 32, 32)
labels = t['targets']

In [ ]:
resnet = models.resnet18(pretrained=True)
resnet.fc = torch.nn.Identity()  # Remove the final layer to get feature vectors
resnet.eval()

In [20]:
images.shape

(2500, 3, 32, 32)

In [21]:
print(labels)

[6 9 9 ... 9 9 7]


In [ ]:
print 